In [1]:
import numpy as np
from scipy.linalg import solve_toeplitz, matmul_toeplitz

In [5]:
class InverseToeplitz:
    """A class for storing a Toeplitz matrix,
    the Gohberg-Semencul decomposition of its inverse,
    and use it to apply the inverse to a vector.

    Methods
    -------
    apply_inverse
    """

    def __init__(self, col: np.ndarray, row=None) -> None:
        """A Toeplitz matrix T is represented by:

        Parameters
        ----------
        col : np.ndarray
            First column of T.
        row : np.ndarray | None
            First row of T. By default None, meaning row = conj(col)


        - its first column: col
        - its first row: row

        If row is not provided, it is assumed T is Hermitian.
        """
        self.col = col
        self.row = row

        self.x0 = None
        self.m1 = None
        self.m2 = None
        self.m3 = None
        self.m4 = None

        self.gohberg_semencul()

    def gohberg_semencul(self) -> None:
        """Computes the Gohberg-Semencul decomposition of T^{-1}"""
        e0 = np.zeros_like(self.col)
        e0[0] = 1

        e1 = np.zeros_like(self.col)
        e1[-1] = 1

        if self.row is None:
            x = solve_toeplitz(self.col, e0)
            y = solve_toeplitz(self.col, e1)

        else:
            x = solve_toeplitz((self.col, self.row), e0)
            y = solve_toeplitz((self.col, self.row), e1)

        x_a = np.zeros_like(x)
        x_a[0] = x[0]

        x_b = np.zeros_like(x)
        x_b[1::] = x[:0:-1]

        y_a = np.zeros_like(y)
        y_a[0] = y[-1]

        y_b = np.zeros_like(y)
        y_b[1::] = y[:-1]

        self.x0 = x[0]
        self.m1 = (x, x_a)
        self.m2 = (y_a, y[::-1])
        self.m3 = (y_b, np.zeros_like(y))
        self.m4 = (np.zeros_like(x), x_b)

    def apply_inverse(self, vec: np.ndarray, workers: int = 8) -> np.ndarray:
        """Computes T^{-1} @ vec using the Gohberg-Semencul formula.

        Parameters
        ----------
        vec : np.ndarray
            Vector to compute the product T^{-1} @ vec.
        workers : int
            Workers parameter for scipy.linalg.matmul_toeplitz, by default 8

        Returns
        -------
        out : np.ndarray
            Value of T^{-1} @ vec
        """
        M1M2_v = matmul_toeplitz(
            self.m1, matmul_toeplitz(self.m2, vec, workers), workers
        )

        M3M4_v = matmul_toeplitz(
            self.m3, matmul_toeplitz(self.m4, vec, workers), workers
        )

        return (M1M2_v - M3M4_v) / self.x0

# Fast version

We first check how to do `matmul_toeplitz` manually.

In [6]:
def my_matmul_toeplitz(c, x, r=None):
    n = len(x)
    if r is None:
        r = np.conjugate(c)

    cr = np.concatenate((r[:0:-1], c)) # size 2n - 1
        
    return np.fft.ifft(np.fft.fft(x, n=3*n-2) * np.fft.fft(cr, n=3*n-2))[n-1:2*n-1]

In [7]:
n = 10
c = np.random.rand(n)
r = np.random.rand(n)
x = np.random.rand(n)
print(np.allclose(my_matmul_toeplitz(c, x, r), matmul_toeplitz((c, r), x)))
print(np.allclose(my_matmul_toeplitz(c, x), matmul_toeplitz(c, x)))

n = 11
c = np.random.rand(n)
r = np.random.rand(n)
x = np.random.rand(n)
print(np.allclose(my_matmul_toeplitz(c, x, r), matmul_toeplitz((c, r), x)))
print(np.allclose(my_matmul_toeplitz(c, x), matmul_toeplitz(c, x)))

True
True
True
True


In [8]:
class FastInverseToeplitz:
    """A class for storing a Toeplitz matrix,
    the Gohberg-Semencul decomposition of its inverse,
    and use it to apply the inverse to a vector.

    Methods
    -------
    apply_inverse
    """

    def __init__(self, col: np.ndarray, row=None) -> None:
        """A Toeplitz matrix T is represented by:

        Parameters
        ----------
        col : np.ndarray
            First column of T.
        row : np.ndarray | None
            First row of T. By default None, meaning row = conj(col)


        - its first column: col
        - its first row: row

        If row is not provided, it is assumed T is Hermitian.
        """
        self.col = col
        self.row = row
        self.n = len(col)

        self.x0 = None
        self.fft_m1 = None
        self.fft_m2 = None
        self.fft_m3 = None
        self.fft_m4 = None

        self.gohberg_semencul()

    def gohberg_semencul(self) -> None:
        """Computes the Gohberg-Semencul decomposition of T^{-1}"""
        e0 = np.zeros_like(self.col)
        e0[0] = 1

        e1 = np.zeros_like(self.col)
        e1[-1] = 1

        if self.row is None:
            x = solve_toeplitz(self.col, e0)
            y = solve_toeplitz(self.col, e1)

        else:
            x = solve_toeplitz((self.col, self.row), e0)
            y = solve_toeplitz((self.col, self.row), e1)

        self.fft_m1 = np.fft.fft(np.concatenate((np.zeros(n-1), x)))

        self.fft_m4 = np.fft.fft(np.concatenate((x[1:], np.zeros(n))))

        col_m2 = np.zeros_like(y)
        col_m2[0] = y[-1]
        self.fft_m2 = np.fft.fft(np.concatenate((y[:-1], col_m2)))

        col_m3 = np.zeros_like(y)
        col_m3[1::] = y[:-1]
        self.fft_m3 = np.fft.fft(np.concatenate((np.zeros(n-1), col_m3)))

        self.x0 = x[0]

    def apply_inverse(self, x: np.ndarray) -> np.ndarray:
        """Computes T^{-1} @ vec using the Gohberg-Semencul formula.
        Uses np.fft 6 times instead of 12.

        Parameters
        ----------
        x : np.ndarray
            Vector to compute the product T^{-1} @ x.
        workers : int
            Workers parameter for scipy.linalg.matmul_toeplitz, by default 8

        Returns
        -------
        out : np.ndarray
            Value of T^{-1} @ x
        """

        fft_x = np.fft.fft(x, n=2*self.n-1)

        m2_x = np.fft.ifft(self.fft_m2 * fft_x)[-self.n:]
        m4_x = np.fft.ifft(self.fft_m4 * fft_x)[-self.n:]

        res = np.fft.ifft(self.fft_m1 * np.fft.fft(m2_x, n=2*self.n-1) - self.fft_m3 * np.fft.fft(m4_x, n=2*self.n-1))[-self.n:]

        return res / self.x0

In [9]:
n = 10
c = np.random.rand(n) + 1j * np.random.rand(n)
x = np.random.rand(n) + 1j * np.random.rand(n)

old_top = InverseToeplitz(c)
new_top = FastInverseToeplitz(c)

np.allclose(old_top.apply_inverse(x), new_top.apply_inverse(x))

[-0.06070225-0.38219609j -0.0441928 +0.2133863j  -0.0695824 +0.13299064j
  0.00784348+0.01905012j  0.02498044-0.02564702j -0.16847488+0.03740223j
  0.21449808+0.14024091j -0.12582266-0.15322444j -0.06383554+0.05421789j
  0.20973099+0.17611828j]
[ 0.24270567+0.19390203j  0.02968738+0.1319017j   0.01694945-0.00899186j
  0.19208448-0.08689078j -0.13590862+0.09832271j  0.26050271-0.07767263j
 -0.04436297-0.01133576j  0.05937902-0.08769083j  0.09408789-0.06865334j
 -0.06070225-0.38219609j]


True

In [25]:
n = 2048
c = np.random.rand(n) + 1j * np.random.rand(n)
x = np.random.rand(n) + 1j * np.random.rand(n)

In [24]:
%timeit old_top.apply_inverse(x)

1.31 ms ± 46.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
%timeit new_top.apply_inverse(x)

531 µs ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
